# BLE Remote Control

The remote controller is used to drive the robot, but also for configuration. It has:

* an analog joystick (read by the ESP32 ADC)
* 3 quadrature encoded potentiometer that also server as push buttons
* 3 LEDs

The ESP32 reports changes over a BLE UART service and also listens for updates, e.g. of the LEDs, using a simple binary protocol. One button acts as "power switch": when off, the ESP32 is put into deepsleep to preserve power when operated from a battery. Pressing the button in this state resumes operation.

The function of the LEDs is as follows:

* RED: esp32 is advertising or connected to a BLE central
* GREEN: connected to a BLE central
* BLUE: *spare*

## Protocol

Messages consist of a one byte (character) code, followed by a binary float (4 bytes).

### MCU send

| code | description |
| ---- | ----------- |
| x,y  | joy-stick position |
|  q   | quit button pressed, ESP32 going into deepsleep |
| b,c  | button b,c pressed count (since powerup) |
| 1,2,3| encoder counts |
|  v   | battery voltage |
|  h   | heartbeat count |

   
### MCU receive

| code | description |
| ---- | ----------- |
| R,G,B| led on/off (1/0). Note: only B (blue) is unassigned. |
|  Q   | instruct ESP32 to go into deepsleep, press power button on remote to wake up |

## Configure ESP32

In [2]:
%connect robot-esp32
%rsync
%rlist
%softreset

Connected to robot-esp32 @ serial:///dev/ttyUSB0
Directories match
     67  Jan 20 15:16 2022  boot.py
                            lib/
   2713  Jan 20 15:16 2022      ble_advertising.py
   1147  Jan 20 15:16 2022      filter.py
   5022  Jan 20 15:16 2022      loop.py
                                remote/
    145  Jan 20 15:16 2022          __init__.py
   2651  Jan 20 15:18 2022          ble_uart.py
    512  Jan 20 15:16 2022          button.py
    922  Jan 20 15:16 2022          encoder.py
    777  Jan 20 15:16 2022          joy_axis.py
    139  Jan 20 15:16 2022  main.py
    317  Jan 20 15:16 2022  secrets.py

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


## Functionality

`loop.py`, called on every boot from `main.py` continuously scans the peripherals (using a few helper classes) and BLE messages.

In [4]:
!cat $IOT_PROJECTS/robot/code/esp32/lib/loop.py

from remote import BLE_UART, JoyAxis, Button, Encoder
from machine import ADC, Pin, deepsleep, reset_cause, PWRON_RESET
from struct import pack, unpack
from time import ticks_ms, ticks_diff, sleep_ms
import esp32

"""
Messages:
--------

Format:
pack('>Bf', ord(code), value)

Codes:

a) MCU -> Pi
   x, y: joystick position
   q, b, c: button pressed counts
         Note: button 1 assigned to 'q' (quit = deepsleep)
   1, 2, 2: encoder counts
   v: v_bat [V]
   h: heartbeat count

b) Pi -> MCU
   R, G, B: led on/off (1/0)
   Q: power down remote (deepsleep)
"""

HARTBEAT_MS =   500     # frequency with which hartbeats are sent [ms]
SHUTDOWN_MS = 60000     # shut down if no connection in specified time [ms]
VBATT_MS    = 60000     # rate at which battery level is sent [ms]

def loop():

    # deepsleep wakeup
    deepsleep_wakeup_pin = Pin(4, mode=Pin.IN, pull=Pin.PULL_UP)
    esp32.wake_on_ext0(deepsleep_wakeup_pin, esp32.WAKEUP_ALL_LOW)

    # BLE
    def rx_cb(data):
        nonlocal r

## Raspberry PI

In [5]:
%%host

import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

import asyncio
from robot import Remote

def exception_handler(loop, context):
    msg = context.get("exception", context["message"])
    print("***** asyncio:", context)
    print("***** msg:", msg)
    
remote = Remote()

asyncio.get_event_loop().set_exception_handler(exception_handler)
asyncio.run(remote.run(peripheral_name='iot49-robot'))

print("DONE!")

scanning for iot49-robot
connecting to iot49-robot ... connected
RECV v =    4.225   dt =        192.2 [ms]
RECV b =    1.000   dt =        487.3 [ms]
RECV x =   -0.142   dt =        729.9 [ms]
RECV x =   -0.081   dt =        48.27 [ms]
RECV 3 =    1.000   dt =        194.8 [ms]
RECV 3 =    2.000   dt =        193.6 [ms]
RECV c =    1.000   dt =        243.1 [ms]
---------- Remote powered down - QUIT!
so long ...
DONE!
